In [ ]:
from ner.Datasets.utils import *
from ner.Datasets.Conll2003Dataset import *

%load_ext autoreload
%autoreload 2

In [ ]:
# data_tr = Conll2003Dataset(utils, split = 'train')
# data_te = Conll2003Dataset(utils, split = 'test')

In [ ]:
clean_data_te = Conll2003Dataset(split = 'test', cleaned = True)
clean_data_tr = Conll2003Dataset(split = 'train', cleaned = True)

In [ ]:
clean_data_te

In [ ]:
save_conll_dataset(clean_data_te)
save_conll_dataset(clean_data_tr)

In [ ]:
dataset_test = load_conll_dataset(cleaned= True)

In [ ]:
dataset_train.dataset

## Preparing datasets for finetuning

In [5]:
from ner.utils import load, dump
from llm.LLMModel import MistralAIInstruct
%reload_ext autoreload
%autoreload 2

model = MistralAIInstruct(without_model=True)

dataset = load("ner/saves/datasets/conll2003_for-ft_cleaned_discussion_2000.pkl")
# dataset[0]['text']
dataset = dataset.map(lambda row : {'text' : model.construct_prompt_for_mistralAIInstruct(row['text']) })
dump(dataset, "ner/saves/datasets/conll2003_for-ft_cleaned_discussion_mistral-instruct_2000.pkl")

Parameter 'function'=<function <lambda> at 0x7f1aed503eb0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
dataset[0]

{'text': '<s>[INST] The task is to extract all the named entites in the following sentence.\nYour goal is to extract all the enities that are either person, organization, location or miscallaneous and output the entities in a list of tuples. In each tuple put the named entity and the tag alongside it.\n [/INST]Yes I can do that. Can you provide me examples ?  \n</s>[INST] Yes of course, there are some examples : \n<start_input> Yr / yr rise ( % ) +1.2 +1.5 +7.1 119.6 - <end_input>\n [/INST]<start_output> [] <end_output>\n\nOk now I understand I need to only output a list with the entities that are in the sentence and the tag along it. Can you now provide me the sentence ? \n</s>[INST] <start_input> PPI ( pct ) June +0.7 m / m;+21.5yr / yr ( May +1.7;+22.0 ) <end_input>\n [/INST]<start_output> [] <end_output></s>'}

In [ ]:
from ner.llm_ner.prompt_techniques import *

runs = 2000
pt = PT_GPT_NER(None)
res2 = pt.process_dataset_for_finetuning(runs=runs)
pt = PT_OutputList(None)
res2 = pt.process_dataset_for_finetuning(runs=runs)
pt = PT_Wrapper(None)
res2 = pt.process_dataset_for_finetuning(runs=runs)

## Testing few-shots

In [ ]:
# dataset_train, dataset_llama_50 = load_conll_dataset(split = 'test')
# dataset_test = load_conll_dataset(split = 'test', cleaned=True)
dataset_test_train, dataset_test_test = dataset_test.train_test_split(test_size=20)

In [ ]:
from ner.Llama2_NER import Llama2_NER_FewShots_Entity
ner_ent = Llama2_NER_FewShots_Entity(None, utils, dataset_test_train, nb_few_shots= 16, with_verify=False, prompt_type="discussion")

In [ ]:
# dataset_llama_50.select([8,10,22,24,25,44,49])
for i, sample in enumerate(dataset_test_test) :
    print(sample['text'])
    print(sample['spans'])
    print(ner_ent.get_prompts(sample['text'], [], 5))
    print()
    print("----------------------------------------------------")
    print()